In [17]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
from dimod import ExactSolver
from dwave.system.composites import EmbeddingComposite
from dwave.system import FixedEmbeddingComposite
import dwavebinarycsp as dbc
from helpers.embedding import embeddings
from helpers import draw

# 0. Connect to AWS

bucket = 'amazon-braket-7481abb5f17b'
prefix = 'dwave'
s3_path = (bucket, prefix)

device = AwsDevice('arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

In [4]:
# 1. Express factoring as multiplication circuit

csp = dbc.factories.multiplication_circuit(3)
print(next(iter(csp.constraints)))

Constraint.from_configurations(frozenset({(1, 0, 0), (0, 0, 0), (1, 1, 1), (0, 1, 0)}), ('a0', 'b0', 'p0'), Vartype.BINARY, name='AND(a0, b0) = p0')


In [5]:
# 2. Convert CSP to BQM

bqm = dbc.stitch(csp, min_classical_gap=.1)

print("BQM has {} variables: \n{}".format(len(bqm.variables), list(bqm.variables)))

BQM has 27 variables: 
['a0', 'b0', 'p0', 'b1', 'and0,1', 'b2', 'and0,2', 'a1', 'and1,0', 'p1', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'a2', 'and2,0', 'p2', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'p3', 'carry3,0', 'p4', 'p5']


In [6]:
P = 35
bP = '{:06b}'.format(P)
p_vars = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5']

fixed_variables = dict(zip(reversed(p_vars), bP))

for var, v in fixed_variables.items():
    bqm.fix_variable(var, int(v))

print("BQM has {} non-fixed variables: \n{}".format(len(bqm.variables), list(bqm.variables)))

BQM has 21 non-fixed variables: 
['a0', 'b0', 'b1', 'and0,1', 'b2', 'and0,2', 'a1', 'and1,0', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'a2', 'and2,0', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'carry3,0']


In [7]:
# 3. Submit to Quantum Computer

sampler = BraketDWaveSampler(s3_path,'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6')

sampler_comp = FixedEmbeddingComposite(sampler, embeddings['DW_2000Q_6'])

print("Mapping from BQM variables to qubits for the Dwave system:\n\n\t{}", sampler_comp.embedding)

Mapping from BQM variables to qubits for the Dwave system:

	{} {'and2,1': (590, 582, 450, 578), 'sum1,1': (469, 464, 592, 720), 'carry1,0': (727, 851, 723), 'and2,2': (577, 449, 455, 448), 'a1': (707, 711, 719), 'and2,0': (709, 717, 725), 'and0,1': (854, 843, 846), 'a0': (585, 841, 713), 'and0,2': (456, 597, 584, 589), 'carry2,1': (457, 454, 462), 'carry2,0': (722, 594, 466), 'sum2,1': (452, 467, 468, 460), 'a2': (710, 704, 576), 'b0': (840, 712, 718), 'b1': (706, 834, 842, 839, 847), 'b2': (579, 583, 591), 'and1,2': (715, 587, 459), 'carry3,0': (471, 463), 'and1,0': (835, 837, 853, 845), 'and1,1': (586, 724, 716, 714), 'carry1,1': (458, 453, 461)}


In [8]:
# import multiprocessing

sampleset = sampler_comp.sample(bqm, num_reads=100)


print("Best solution found: \n", sampleset.first.sample)

Best solution found: 
 {'a0': 1, 'a1': 0, 'a2': 1, 'and0,1': 1, 'and0,2': 1, 'and1,0': 0, 'and1,1': 0, 'and1,2': 0, 'and2,0': 1, 'and2,1': 1, 'and2,2': 1, 'b0': 1, 'b1': 1, 'b2': 1, 'carry1,0': 0, 'carry1,1': 0, 'carry2,0': 1, 'carry2,1': 0, 'carry3,0': 1, 'sum1,1': 1, 'sum2,1': 1}


In [13]:
from helpers.convert import to_base_ten

a, b = to_base_ten(sampleset.first.sample)

print("solution: {} = {}*{}".format(P, a, b))

solution: 35 = 5*7


In [14]:
from collections import OrderedDict

def response_to_dict(sampleset):
    results_dict = OrderedDict()
    for sample, energy in sampleset.data(['sample', 'energy']):
        a, b = to_base_ten(sample)
        if (a, b) not in results_dict:
            results_dict[(a, b)] = round(energy, 2)
            
    return results_dict

results = response_to_dict(sampleset)
results

OrderedDict([((5, 7), -0.0),
             ((7, 5), -0.0),
             ((7, 7), 2.0),
             ((7, 3), 2.0),
             ((3, 7), 2.0),
             ((3, 5), 2.0),
             ((5, 3), 2.0),
             ((5, 5), 2.0),
             ((7, 6), 3.0),
             ((6, 3), 3.0),
             ((3, 6), 3.0),
             ((3, 3), 3.0),
             ((1, 7), 3.0),
             ((7, 1), 3.0),
             ((3, 1), 3.0),
             ((1, 3), 6.0)])

In [18]:
draw.energy_of(results)